In [1]:
from query.query_engine import create_query_engine
from query.routing import create_routing_engine
from evaluation.eval_with_llamaindex import evaluate, evaluate_multiple_experiments

from loguru import logger
from pprint import pprint
import pandas as pd
import nest_asyncio
nest_asyncio.apply()
import sys
import os

#set level of logging
logger.remove()
logger.add(sys.stderr, level="INFO")
os.chdir("..")

In [15]:
def create_final_csv(scores_df: pd.DataFrame) -> pd.DataFrame:
    scores_df.rename(columns={"mistral_embed": "mistral-embed", 
                              "fastembed": "multilingual-e5-large",
                              "ada": "text-embedding-ada-002"}, 
                     inplace=True)
    
    embedding_times = [["3m", "22m",  "1m"]]
    
    # add row with embeddings times and the index name is "embedding_time"
    df_embedding_times = pd.DataFrame(embedding_times, index = ["embedding_time"], columns=scores_df.columns)
    
    final_df = pd.concat([df_embedding_times, scores_df])
    
    final_df = final_df.rename_axis('embeddings', axis='columns')

    
    return final_df

# Embeddings evaluation

In [3]:
query_engine_fastembed = create_query_engine(embedding_model="intfloat/multilingual-e5-large")
query_engine_mistral = create_query_engine(embedding_model="mistral-embed")
query_engine_ada = create_query_engine(embedding_model="text-embedding-ada-002")

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

2024-03-24 17:45:23.676 | INFO     | retriever.get_retriever:index_nodes:82 - Creating text nodes ...
2024-03-24 17:45:23.696 | INFO     | data_ingestion.nodes_processing:_truncate_long_articles:119 - Truncated 0 articles into smaller chunks.
2024-03-24 17:45:23.743 | INFO     | retriever.get_retriever:index_nodes:93 - Text nodes creation finished.
2024-03-24 17:45:23.744 | INFO     | retriever.get_retriever:index_given_nodes:24 - Collection name: Code_civil_base_intfloat_multilingual-e5-large
2024-03-24 17:45:23.781 | WARNING  | retriever.get_retriever:index_given_nodes:33 - Error while counting nodes in the collection : Unexpected Response: 404 (Not Found)
Raw response content:
b'{"status":{"error":"Not found: Collection `Code_civil_base_intfloat_multilingual-e5-large` doesn\'t exist!"},"time":0.0004615}'Setting count to 0.
2024-03-24 17:45:23.781 | INFO     | retriever.get_retriever:index_given_nodes:51 - Creating a new index with 2882 nodes. This may take a while.
2024-03-24 18:07:

In [6]:
exp_to_query_engine = {
    "mistral_embed": query_engine_mistral,
    "fastembed": query_engine_fastembed,
    "ada": query_engine_ada,
}

scores_df, deeps_df = evaluate_multiple_experiments(
    experiment_to_query_engine=exp_to_query_engine,
    general_exp_name="embeddings",
    list_metrics=["context_relevancy"],
    code_name="code_civil"
)


2024-03-24 20:09:09.524 | INFO     | evaluation.eval_with_llamaindex:evaluate_multiple_experiments:130 - Evaluating experiment: mistral_embed
100%|██████████| 50/50 [04:25<00:00,  5.31s/it]
2024-03-24 20:13:35.209 | INFO     | evaluation.eval_with_llamaindex:evaluate_multiple_experiments:130 - Evaluating experiment: fastembed
100%|██████████| 50/50 [04:01<00:00,  4.84s/it]
2024-03-24 20:17:37.222 | INFO     | evaluation.eval_with_llamaindex:evaluate_multiple_experiments:130 - Evaluating experiment: ada
100%|██████████| 50/50 [04:11<00:00,  5.04s/it]


In [18]:
final_df = create_final_csv(scores_df)
if not os.path.exists("./data/final_results"):
    os.makedirs("./data/final_results")
final_df.to_csv("./data/final_results/embeddings_experiment.csv")

#  Nodes window

## 1. Evaluate context relevancy

In [2]:
query_engine_window_1 = create_query_engine(use_window_nodes=True, nodes_window_size=1, recreate_collection=True)
query_engine_window_3 = create_query_engine(use_window_nodes=True, nodes_window_size=3, recreate_collection=True)

2024-03-24 22:12:11.136 | INFO     | retriever.get_retriever:index_nodes:82 - Creating text nodes ...
2024-03-24 22:12:11.161 | INFO     | data_ingestion.nodes_processing:_truncate_long_articles:119 - Truncated 0 articles into smaller chunks.
2024-03-24 22:12:11.217 | INFO     | data_ingestion.nodes_processing:__post_init__:32 - Adding window nodes ...
Adding window nodes ...: 100%|██████████| 2882/2882 [00:00<00:00, 81041.47it/s]
2024-03-24 22:12:11.268 | INFO     | retriever.get_retriever:index_nodes:93 - Text nodes creation finished.
2024-03-24 22:12:11.269 | INFO     | retriever.get_retriever:index_given_nodes:24 - Collection name: Code_civil_window_text-embedding-ada-002
2024-03-24 22:12:11.379 | WARNING  | retriever.get_retriever:index_given_nodes:33 - Error while counting nodes in the collection : Unexpected Response: 404 (Not Found)
Raw response content:
b'{"status":{"error":"Not found: Collection `Code_civil_window_text-embedding-ada-002` doesn\'t exist!"},"time":0.000073334}'

In [6]:
from llama_index.core.schema import MetadataMode

experiment_to_query_engine = {
    "window_1": query_engine_window_1,
    "window_3": query_engine_window_3,
}
mean_scores_df, deep_dfs = evaluate_multiple_experiments(
    experiment_to_query_engine=experiment_to_query_engine,
    list_metrics=["context_relevancy"],
    general_exp_name="window",
    code_name="code_civil",
    metadata_mode=MetadataMode.NONE,
)


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [04:40<00:00,  5.60s/it]


## 2. evaluate faithfulness

In [2]:
query_engine_base = create_query_engine()
query_engine_window_3 = create_query_engine(use_window_nodes=True, nodes_window_size=3)

2024-03-31 21:08:16.861 | INFO     | retriever.get_retriever:index_nodes:97 - Creating text nodes ...
2024-03-31 21:08:16.890 | INFO     | data_ingestion.nodes_processing:_chunk_long_articles:119 - Truncated 0 articles into smaller chunks.
2024-03-31 21:08:16.941 | INFO     | retriever.get_retriever:index_nodes:108 - Text nodes creation finished.
2024-03-31 21:08:16.942 | INFO     | retriever.get_retriever:index_given_nodes:39 - Collection name: Code_civil_base_text-embedding-ada-002
2024-03-31 21:08:16.965 | INFO     | retriever.get_retriever:index_given_nodes:46 - Found 2882 nodes in the collection.
2024-03-31 21:08:16.966 | INFO     | retriever.get_retriever:index_given_nodes:54 - Found 2882 nodes in the collection. Using the existing collection.
2024-03-31 21:08:17.213 | INFO     | retriever.get_retriever:index_nodes:97 - Creating text nodes ...
2024-03-31 21:08:17.228 | INFO     | data_ingestion.nodes_processing:_chunk_long_articles:119 - Truncated 0 articles into smaller chunks.


In [3]:
experiment_to_query_engine = {
    "base": query_engine_base,
    "window_3": query_engine_window_3,
}
mean_scores_df, deep_dfs = evaluate_multiple_experiments(
    experiment_to_query_engine=experiment_to_query_engine,
    list_metrics=["faithfulness"],
    general_exp_name="window_faithfulness",
    code_name="code_civil",
)

2024-03-31 21:08:23.929 | INFO     | evaluation.eval_with_llamaindex:evaluate_multiple_experiments:142 - Evaluating experiment: base
100%|██████████| 50/50 [02:27<00:00,  2.95s/it]
2024-03-31 21:10:51.628 | INFO     | evaluation.eval_with_llamaindex:evaluate_multiple_experiments:142 - Evaluating experiment: window_3
100%|██████████| 50/50 [04:41<00:00,  5.63s/it]


# Hybrid search

In [2]:
query_engine_hybrid_search_alpha_0_5 = create_query_engine(hybrid_search=True, sparse_top_k=5)
query_engine_hybrid_search_alpha_0_2 = create_query_engine(hybrid_search=True, sparse_top_k=5, hybrid_search_alpha=0.2)
query_engine_hybrid_search_alpha_0_8 = create_query_engine(hybrid_search=True, sparse_top_k=5, hybrid_search_alpha=0.8)

2024-04-01 11:32:57.392 | INFO     | retriever.get_retriever:index_nodes:97 - Creating text nodes ...
2024-04-01 11:32:57.410 | INFO     | data_ingestion.nodes_processing:_chunk_long_articles:119 - Truncated 0 articles into smaller chunks.
2024-04-01 11:32:57.453 | INFO     | retriever.get_retriever:index_nodes:108 - Text nodes creation finished.
2024-04-01 11:32:57.454 | INFO     | retriever.get_retriever:index_given_nodes:39 - Collection name: Code_civil_base_hybrid_text-embedding-ada-002
2024-04-01 11:32:57.466 | INFO     | retriever.get_retriever:index_given_nodes:46 - Found 2882 nodes in the collection.
2024-04-01 11:32:57.467 | INFO     | retriever.get_retriever:index_given_nodes:54 - Found 2882 nodes in the collection. Using the existing collection.


time : 27 minutes

In [7]:

mean_scores_df, deep_dfs, _ = evaluate(
    query_engine=query_engine_hybrid_search_alpha_0_8,
    list_metrics=["faithfulness", "context_relevancy"],
    exp_name="hybrid_search",
    code_name="code_civil",
)

100%|██████████| 50/50 [03:44<00:00,  4.49s/it]


# Query rewriting

In [2]:
query_engine_base = create_query_engine(hybrid_search=True, 
                                     sparse_top_k=5, 
                                     hybrid_search_alpha=0.8)

query_engine_rewriter = create_query_engine(query_rewrite=True, 
                                     hybrid_search=True, 
                                     sparse_top_k=5, 
                                     hybrid_search_alpha=0.8)

2024-04-01 16:42:26.808 | INFO     | retriever.get_retriever:index_nodes:96 - Creating text nodes ...
2024-04-01 16:42:26.826 | INFO     | data_ingestion.nodes_processing:_chunk_long_articles:102 - Truncated 0 articles into smaller chunks.
2024-04-01 16:42:26.869 | INFO     | retriever.get_retriever:index_nodes:106 - Text nodes creation finished.
2024-04-01 16:42:26.869 | INFO     | retriever.get_retriever:index_given_nodes:39 - Collection name: Code_civil_base_hybrid_text-embedding-ada-002
2024-04-01 16:42:26.884 | INFO     | retriever.get_retriever:index_given_nodes:46 - Found 2882 nodes in the collection.
2024-04-01 16:42:26.885 | INFO     | retriever.get_retriever:index_given_nodes:54 - Found 2882 nodes in the collection. Using the existing collection.
2024-04-01 16:42:28.701 | INFO     | retriever.get_retriever:index_nodes:96 - Creating text nodes ...
2024-04-01 16:42:28.717 | INFO     | data_ingestion.nodes_processing:_chunk_long_articles:102 - Truncated 0 articles into smaller c

In [3]:
experiment_to_query_engine = {
    "base": query_engine_base,
    "rewriting_query": query_engine_rewriter,
}
mean_scores_df, deep_dfs = evaluate_multiple_experiments(
    experiment_to_query_engine=experiment_to_query_engine,
    list_metrics=["faithfulness", "answer_relevancy"],
    general_exp_name="query_rewriting",
    code_name="code_civil",
)

2024-04-01 16:42:33.290 | INFO     | evaluation.eval_with_llamaindex:evaluate_multiple_experiments:152 - Evaluating experiment: base
100%|██████████| 50/50 [03:25<00:00,  4.12s/it]
2024-04-01 16:48:21.930 | INFO     | evaluation.eval_with_llamaindex:evaluate_multiple_experiments:152 - Evaluating experiment: rewriting_query
100%|██████████| 50/50 [05:38<00:00,  6.77s/it]


# Routing

In [2]:
routing_query_engine = create_routing_engine()

2024-04-01 21:16:32.315 | INFO     | retriever.get_retriever:index_nodes:96 - Creating text nodes ...
2024-04-01 21:16:32.337 | INFO     | data_ingestion.nodes_processing:_chunk_long_articles:102 - Truncated 0 articles into smaller chunks.
2024-04-01 21:16:32.384 | INFO     | retriever.get_retriever:index_nodes:106 - Text nodes creation finished.
2024-04-01 21:16:32.385 | INFO     | retriever.get_retriever:index_given_nodes:39 - Collection name: Code_civil_base_text-embedding-ada-002
2024-04-01 21:16:32.418 | INFO     | retriever.get_retriever:index_given_nodes:46 - Found 2882 nodes in the collection.
2024-04-01 21:16:32.418 | INFO     | retriever.get_retriever:index_given_nodes:54 - Found 2882 nodes in the collection. Using the existing collection.
2024-04-01 21:16:32.622 | INFO     | retriever.get_retriever:index_nodes:96 - Creating text nodes ...
2024-04-01 21:16:32.640 | INFO     | data_ingestion.nodes_processing:_chunk_long_articles:102 - Truncated 2 articles into smaller chunks.


# other

In [2]:
query_rewriter = create_query_engine(query_rewrite=True, 
                                     hybrid_search=True, 
                                     sparse_top_k=5, 
                                     hybrid_search_alpha=0.8)

2024-04-01 16:20:11.357 | INFO     | retriever.get_retriever:index_nodes:97 - Creating text nodes ...
2024-04-01 16:20:11.375 | INFO     | data_ingestion.nodes_processing:_chunk_long_articles:119 - Truncated 0 articles into smaller chunks.
2024-04-01 16:20:11.418 | INFO     | retriever.get_retriever:index_nodes:108 - Text nodes creation finished.
2024-04-01 16:20:11.419 | INFO     | retriever.get_retriever:index_given_nodes:39 - Collection name: Code_civil_base_hybrid_text-embedding-ada-002
2024-04-01 16:20:11.436 | INFO     | retriever.get_retriever:index_given_nodes:46 - Found 2882 nodes in the collection.
2024-04-01 16:20:11.437 | INFO     | retriever.get_retriever:index_given_nodes:54 - Found 2882 nodes in the collection. Using the existing collection.


In [3]:
query_engine_rewriter.query("Est-ce que je peux refuser un héritage?")

Generated queries:
1. Quelles sont les procédures légales pour refuser un héritage selon le Code civil français?
2. Quels sont les droits et obligations d'un héritier qui souhaite renoncer à une succession en France?
3. Comment se déroule la renonciation à un héritage en vertu du Code civil français?


Response(response="Selon le code civil et l'article 804, la renonciation à une succession ne se présume pas.", source_nodes=[NodeWithScore(node=TextNode(id_='a7e91b29-16a6-46ff-a755-45d7c8ea7b26', embedding=None, metadata={'Nom du code': 'Code civil', 'livre': 'Des différentes manières dont on acquiert la propriété', 'titre': 'Des successions', 'chapitre': "De l'option de l'héritier", 'section': 'De la renonciation à la succession.', 'id': '804', 'Article numero': '804'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="La renonciation à une succession ne se présume pas.\nPour être opposable aux tiers, la renonciation opérée par l'héritier universel ou à titre universel doit être adressée ou déposée au tribunal dans le ressort duquel la succession s'est ouverte ou faite devant notaire.\nDans le mois suivant la renonciation, le notaire qui l'a reçue en adresse copie au tribunal dans le ressort duquel la succession s'est ouverte.", start_char_idx=No

In [9]:
from llama_index.core.evaluation import FaithfulnessEvaluator
from llama_index.llms.openai import OpenAI
gpt4 = OpenAI(temperature=0, model="gpt-4")

evaluator_gpt4 = FaithfulnessEvaluator(llm=gpt4)

In [10]:
response_vector = query_engine_window.query("How did New York City get its name?")
eval_result = evaluator_gpt4.evaluate_response(response=response_vector)

In [11]:
eval_result

EvaluationResult(query=None, contexts=["Nom du code: Code civil\ntitre: De la publication, des effets et de l'application des lois en général\nid: 2\nArticle numero: 2\n\nLa loi ne dispose que pour l'avenir ; elle n'a point d'effet rétroactif.\nNom du code: Code civil\ntitre: De la publication, des effets et de l'application des lois en général\nid: 3\nArticle numero: 3\n\nLes lois de police et de sûreté obligent tous ceux qui habitent le territoire.\nLes immeubles, même ceux possédés par des étrangers, sont régis par la loi française.\nLes lois concernant l'état et la capacité des personnes régissent les Français, même résidant en pays étranger.\nNom du code: Code civil\ntitre: De la publication, des effets et de l'application des lois en général\nid: 4\nArticle numero: 4\n\nLe juge qui refusera de juger, sous prétexte du silence, de l'obscurité ou de l'insuffisance de la loi, pourra être poursuivi comme coupable de déni de justice.", "Nom du code: Code civil\nlivre: Des différentes m

In [3]:
resp = query_engine_window.query("art 1")

In [4]:
import string
content = resp.source_nodes[4].get_content()
content.count("Nom du code")

3

In [5]:
a = [2, 3, 4, 5, 6, 7, 8, 9, 10]

a[1-1:1+1]

[2, 3]

| Month    | Savings |
| -------- | ------- |
| January  | $250    |
| February | $80     |
| March    | $420    |

In [3]:
out = evaluate(query_engine=query_engine, list_metrics=["context_relevancy"], exp_name="basic", code_name="code_civil")

  0%|          | 0/2 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 2/2 [00:11<00:00,  5.90s/it]


In [5]:
out["CONTEXT_RELEVANCY"]

,rag,query,answer,contexts,scores,feedbacks
0,basic,Quelles sont les conditions requises pour qu'u...,None,[Le présent code est applicable à Mayotte dans...,0.125,1. The retrieved context does not match the su...
1,basic,Comment peut-on établir la filiation d'un enfa...,None,[Le titre de propriété et ses inscriptions con...,0.000,1. The retrieved context does not match the su...


In [6]:
df = pd.read_csv("data/evaluation_results/basic_code_civil_metrics.csv")

In [5]:
df

,Unnamed: 0,rag,query,answer,contexts,scores,feedbacks,metric
0,0,basic,Quelles sont les conditions requises pour qu'u...,NaN,['Le présent code est applicable à Mayotte dan...,0.0,1. The retrieved context does not match the su...,context_relevancy
1,1,basic,Comment peut-on établir la filiation d'un enfa...,NaN,"[""Le titre de propriété et ses inscriptions co...",0.0,1. The retrieved context does not match the su...,context_relevancy


In [7]:
df

,metrics,basic
0,mean_context_relevancy_score,0.0
